In [25]:
import requests
import PyPDF2
import pandas as pd

In [26]:
from datasets import load_dataset
dataset = load_dataset("PatronusAI/financebench")

In [27]:
df = pd.DataFrame(dataset['train'])

In [28]:
df['doc_link'][0]

'https://investors.3m.com/financials/sec-filings/content/0001558370-19-000470/0001558370-19-000470.pdf'

# I need to create a mapping from file paths to the correct instances(rows) so that I can perform testing.

In [5]:
urls = df['doc_link']

In [6]:
urls

0      https://investors.3m.com/financials/sec-filing...
1      https://investors.3m.com/financials/sec-filing...
2      https://investors.3m.com/financials/sec-filing...
3      https://investors.3m.com/financials/sec-filing...
4      https://investors.3m.com/financials/sec-filing...
                             ...                        
145    https://www.verizon.com/about/sites/default/fi...
146    https://www.verizon.com/about/sites/default/fi...
147    https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...
148    https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...
149    https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...
Name: doc_link, Length: 150, dtype: object

In [7]:
paths = []
for url in urls:
    if url.endswith('.pdf'):
        file_name = url.split("/")[-1]
        save_path = f"financial_statements/{file_name}" 
        paths.append(save_path)
    else: 
        file_name = url.split("/")[-1]
        file_name = file_name + '.pdf'
        save_path = f"financial_statements/{file_name}" 
        paths.append(save_path)



In [8]:
df['financial_pdf_path'] = paths #Creating column 

In [9]:
df['financial_pdf_path']

0          financial_statements/0001558370-19-000470.pdf
1          financial_statements/0001558370-19-000470.pdf
2          financial_statements/0000066740-23-000014.pdf
3          financial_statements/0000066740-23-000014.pdf
4          financial_statements/0000066740-23-000014.pdf
                             ...                        
145    financial_statements/2022-Annual-Report-on-For...
146    financial_statements/2022-Annual-Report-on-For...
147    financial_statements/a25e7acb-aa07-49f3-8c0c-0...
148    financial_statements/b23b2787-eb44-4e0b-82bd-f...
149    financial_statements/af5415d9-0e07-4ba1-a6cc-b...
Name: financial_pdf_path, Length: 150, dtype: object

# Filtering these incorrect rows

In [10]:
df[df['financial_pdf_path'].str.contains('pdfTarget')]


,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number,financial_pdf_path
10,financebench_id_04735,ADOBE_2015_10K,https://www.adobe.com/pdf-page.html?pdfTarget=...,2015,metrics-generated,You are an investment banker and your only res...,0.66,59\n ADOBE SYSTEMS INCORPORATED\n CONSOLIDATED...,"59,63",financial_statements/pdf-page.html?pdfTarget=a...
11,financebench_id_07507,ADOBE_2016_10K,https://www.adobe.com/pdf-page.html?pdfTarget=...,2016,metrics-generated,What is Adobe's year-over-year change in unadj...,65.4%,Table of Contents\n62\nADOBE SYSTEMS INCORPORA...,62,financial_statements/pdf-page.html?pdfTarget=a...
12,financebench_id_03856,ADOBE_2017_10K,https://www.adobe.com/pdf-page.html?pdfTarget=...,2017,metrics-generated,What is the FY2017 operating cash flow ratio f...,0.83,Table of Contents\n57\n ADOBE SYSTEMS INCORPOR...,"57,61",financial_statements/pdf-page.html?pdfTarget=a...
13,financebench_id_00438,ADOBE_2022_10K,https://www.adobe.com/pdf-page.html?pdfTarget=...,2022,domain-relevant,Does Adobe have an improving operating margin ...,No the operating margins of Adobe have recentl...,ADOBE INC.\nCONSOLIDATED STATEMENTS OF INCOME\...,54,financial_statements/pdf-page.html?pdfTarget=a...
14,financebench_id_00591,ADOBE_2022_10K,https://www.adobe.com/pdf-page.html?pdfTarget=...,2022,novel-generated,Does Adobe have an improving Free cashflow con...,"Yes, the FCF conversion (using net income as t...",ADOBE INC.\n CONSOLIDATED STATEMENTS OF CASH F...,57,financial_statements/pdf-page.html?pdfTarget=a...


In [11]:
df[df['financial_pdf_path'].str.contains('q1-2023-earning')]


,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number,financial_pdf_path
128,financebench_id_01474,PEPSICO_2023Q1_EARNINGS,https://investors.pepsico.com/docs/default-sou...,2023,novel-generated,"As of FY2023Q1, why did Pepsico raise full yea...",Pepsico experienced a strong start to FY2023.,“We are very pleased with our performance and ...,1,financial_statements/q1-2023-earnings-release_...
129,financebench_id_01476,PEPSICO_2023Q1_EARNINGS,https://investors.pepsico.com/docs/default-sou...,2023,novel-generated,"As of FY2023Q1, by how many percentage points ...",Pepsico raised full year guidance in respect o...,“We are very pleased with our performance and ...,1,financial_statements/q1-2023-earnings-release_...


# Making changes for these

In [12]:
condition1 = df['financial_pdf_path'].str.contains('pdfTarget', case=False, na=False)


In [13]:
condition2 = df['financial_pdf_path'].str.contains('q1-2023-earning', case=False, na=False)


In [14]:
df.loc[condition1, 'financial_pdf_path'] = 'financial_statements/ADBE-10K-FY16-FINAL.pdf'
df.loc[condition2, 'financial_pdf_path'] = 'financial_statements/q1-2023-earnings-release_wfvmt94lzqbm92aw.pdf'

In [15]:
df['financial_pdf_path']

0          financial_statements/0001558370-19-000470.pdf
1          financial_statements/0001558370-19-000470.pdf
2          financial_statements/0000066740-23-000014.pdf
3          financial_statements/0000066740-23-000014.pdf
4          financial_statements/0000066740-23-000014.pdf
                             ...                        
145    financial_statements/2022-Annual-Report-on-For...
146    financial_statements/2022-Annual-Report-on-For...
147    financial_statements/a25e7acb-aa07-49f3-8c0c-0...
148    financial_statements/b23b2787-eb44-4e0b-82bd-f...
149    financial_statements/af5415d9-0e07-4ba1-a6cc-b...
Name: financial_pdf_path, Length: 150, dtype: object

In [16]:
condition2

0      False
1      False
2      False
3      False
4      False
       ...  
145    False
146    False
147    False
148    False
149    False
Name: financial_pdf_path, Length: 150, dtype: bool

# Save to csv

In [17]:
df.to_csv('train.csv',index=False)

In [18]:
df['doc_link'].iloc[106]

'https://s22.q4cdn.com/513010314/files/doc_financials/2022/q4/r/MGM-Resorts-Exhibit-99.1.pdf'

In [19]:
df.loc[106, 'financial_pdf_path'] = "financial_statements/MGM-Resorts-Exhibit-99.1.pdf"

In [20]:
df.loc[106]

financebench_id                                   financebench_id_00382
doc_name                                     MGMRESORTS_2022Q4_EARNINGS
doc_link              https://s22.q4cdn.com/513010314/files/doc_fina...
doc_period                                                         2022
question_type                                           novel-generated
question              Which region had the Highest EBITDAR Contribut...
answer                Las Vegas resorts contributed ~90% of company ...
evidence_text         dited)\nThree months ended Twelve months ended...
page_number                                                          13
financial_pdf_path    financial_statements/MGM-Resorts-Exhibit-99.1.pdf
Name: 106, dtype: object

In [24]:
df['financial_pdf_path'], df['doc_link']

(0          financial_statements/0001558370-19-000470.pdf
 1          financial_statements/0001558370-19-000470.pdf
 2          financial_statements/0000066740-23-000014.pdf
 3          financial_statements/0000066740-23-000014.pdf
 4          financial_statements/0000066740-23-000014.pdf
                              ...                        
 145    financial_statements/2022-Annual-Report-on-For...
 146    financial_statements/2022-Annual-Report-on-For...
 147    financial_statements/a25e7acb-aa07-49f3-8c0c-0...
 148    financial_statements/b23b2787-eb44-4e0b-82bd-f...
 149    financial_statements/af5415d9-0e07-4ba1-a6cc-b...
 Name: financial_pdf_path, Length: 150, dtype: object,
 0      https://investors.3m.com/financials/sec-filing...
 1      https://investors.3m.com/financials/sec-filing...
 2      https://investors.3m.com/financials/sec-filing...
 3      https://investors.3m.com/financials/sec-filing...
 4      https://investors.3m.com/financials/sec-filing...
                 